### 요약 및 키워드 추출 클래스

In [1]:
import threading
from tqdm import tqdm
from pororo import Pororo
from database.controller import ServiceDBController
from elasticsearch import Elasticsearch
import unicodedata as unicode

def summarize_keyword(text_list : list, keyword_news_list : list, news_summary_list : list, lock, keyword_lock):
    
    summ = Pororo(task="summarization", model="extractive", lang="ko")
    ner = Pororo(task="ner", lang="ko")
    pos = Pororo(task="pos", lang="ko")
    es = Elasticsearch("http://search.newsboycorp.com:9200")
    tmp_dbs = ServiceDBController()
    
    error_count = 0
    
    for text in tqdm(text_list):

        
        ##### summarize #####
        try:
            summarized_text = summ(text.get("article_context"))
        except:
            article = text.get("article_context")
            summarized_text = ". ".join(article.split(". ")[:3])
            summarized_text += "...(후략)"

            error_count += 1
        
        ##### elastic search add #####
        id = text.get("id")
        doc = {'title' : text.get('article_headline'), 'summary' : summarized_text, 'article_url' : text.get('article_url'), 'image_url' : text.get('image_url'), "date": text.get("created_datetime")}

        lock.acquire()
#         try:
#             es.create(index="newsboy_news", id=id, body=doc)
#         except:
#             pass
        print(f"ES INSERT : {doc}")
        lock.release()

        ##### extract keyword #####
        keywords = set()
        if len(summarized_text) > 512:
            summarized_text_keyword = summarized_text[:512]
        else:
            summarized_text_keyword = summarized_text
        
        try:
            ner_analysis = ner(summarized_text_keyword)
        except:
            ner_analysis = []
            # continue

        flag = True
        for word, tp in ner_analysis:
            if tp not in ['DATE', 'QUANTITY', 'O', 'COUNTRY', 'LOCATION', 'OCCUPATION', 'TIME', 'CITY', 'CIVILIZATION', 'TERM', 'ANIMAL'] and len(word) > 1:
                if word.isalnum() and (word not in STOPWORDS):
                    # keywords.add((word, tp))
                    for c in word:
                        if 'HANGUL' not in unicode.name(c) and 'LATIN' not in unicode.name(c) and 'DIGIT' not in unicode.name(c):
                            flag = False
                            break
                    if flag:
                        keywords.add((word, tp))
                        flag = True
        
        for wordlist, tp in [(pos(word), tp) for word, tp in list(keywords)]: 
            if len(wordlist) == 1:
                if len(wordlist[0]) == 0 or wordlist[0][1] != 'NNP':
                    keywords.remove((wordlist[0][0], tp))
        


        # upsert keyword on keyword table    
        upsert_keyword_list = []
        for keyword, tp in keywords:
            now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            upsert_keyword_list.append((now, now, keyword, tp, now, keyword,))
        
#         keyword_lock.acquire()
#         tmp_dbs.upsert_keyword(upsert_keyword_list)
#         keyword_lock.release()
        print(f"KEYWORD UPSERT : {upsert_keyword_list}")

        # save relation of keyword and article
        now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        keyword = list(map(lambda x : "\'" + x[0] + "\'", keywords))
        if len(keyword) != 0:
            keyword_news_relation_list = [(now, now, row['keyword_id'], text.get("id"), now) for row in tmp_dbs.get_keyword_ids(", ".join(keyword))] 
        else:
            keyword_news_relation_list = []
        # tmp_dbs.insert_keyword_news_relation(keyword_news_relation_list)

        # lock
        lock.acquire()
        keyword_news_list += keyword_news_relation_list
        news_summary_list.append(
            (text.get("id"), text.get("created_datetime").strftime('%Y-%m-%d %H:%M:%S'), now, summarized_text, text.get("article_url"), 
             text.get("category"), text.get("article_headline"), text.get("image_url"), text.get("score"), now, text.get("score"), )
        )
        lock.release()


### =======================

In [2]:
import datetime
from utils import *
from clustering.dbscan import DBSCAN_news
from database.controller import DBController, ServiceDBController

# time setting
date ='2020-01-10 13:05:00'
file_date = date[:10]
date = datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S')
delta_hour = 1
interval = datetime.timedelta(hours=delta_hour)

# date = datetime.datetime.now()
# file_date = date[:10]
# interval = datetime.timedelta(minutes=NEWS_TIME_INTERVAL)

# settings
NEWS_TIME_INTERVAL = 5
TMP_VECTORIZED_NEWS = f"{file_date}_vectorized_news.pkl"

In [3]:
TMP_VECTORIZED_NEWS

'2020-01-10_vectorized_news.pkl'

In [4]:
def cos_sim(A, B):
    return np.dot(A, B)/(np.linalg.norm(A)*np.linalg.norm(B))

### 새로운 데이터 불러오기

In [5]:
db = DBController()
dbs = ServiceDBController()

In [6]:
recent_news_tmp = db.get_article_by_realtime(date, interval)

recent_news_data = dict()
for news in recent_news_tmp:
    if len(news['article_context'].split('. ')) > 8:
        recent_news_data[news['id']] = dict(news)

### 이전 데이터 불러오기

In [7]:

try:
    prev_news_dict = DataLoader.load_pickle_as_object(TMP_VECTORIZED_NEWS)
except FileNotFoundError:
    prev_news_dict = dict()
    

### 데이터 병합 및 필터링

In [8]:
new_news_list = []
duplicate_list = []
cnt = 1
for k, v in recent_news_data.items():
    if prev_news_dict.get(k) == None:
        new_news_list.append(v)
    else:
        duplicate_list.append(v)

In [9]:
len(new_news_list), len(prev_news_dict.values()), len(duplicate_list)

(461, 0, 0)

### 새로운 데이터 벡터화

In [10]:
%%time
recent_news_dict = Vectorizer.set_df_thread_pool(new_news_list, 8)

CPU times: user 4min 4s, sys: 51.3 s, total: 4min 55s
Wall time: 2min 35s


### 전체 데이터 병합 및 클러스터링

In [11]:
all_data = prev_news_dict
all_data.update(recent_news_dict)

In [12]:
record_num = len(all_data.keys())

In [13]:
DataLoader.save_object_as_pickle(all_data, TMP_VECTORIZED_NEWS)

True

In [14]:
cluster = DBSCAN_news(list(all_data.values()), already_vectorized=True)
cluster_num = cluster.info()['cluster_num'] + cluster.info()['noise_num']

In [15]:
# minimize too big cluster
cluster_id_set = []

max_cnt = 300
eps = 0.15

In [16]:
while True:
    cluster.dataframe["id"] = list(cluster.dataframe.index)
    total_data = cluster.dataframe.to_dict("index")

    # edit eps to minimize cluster
    eps -= 0.04
    if eps < 0.02:
        break

    flag = True

    for cid in range(cluster_num):
        tmp = cluster.dataframe[cluster.dataframe.clusters == cid]
        # over max cluster size
        if len(tmp) > max_cnt:
            flag = False
            tmp_dict = tmp.to_dict(orient = 'records')

            tmp_cluster = DBSCAN_news(tmp_dict, eps=eps, already_vectorized=True)

            # cluster id 가 겹치지 않게 설정
            tmp_cluster.dataframe["clusters"] += cluster_num*(cid+1)

            tmp_data = tmp_cluster.dataframe.to_dict("index")

            for k, v in tmp_data.items():
                total_data[k] = v

    # relocate cluster id 
    cluster_id_set = list(set([v["clusters"] for v in total_data.values()]))

    for v in total_data.values():
        v["clusters"] = cluster_id_set.index(v["clusters"])

    cluster.dataframe = pd.DataFrame.from_dict(total_data, "index")
    cluster_num = len(cluster_id_set)

    if flag:
        break

for k, v in total_data.items():
    v["id"] = k
    v["vector"] = cluster.dataframe.loc[k, "vector"]

df = pd.DataFrame.from_dict(total_data, "index")
df["cos_sim"] = None


/home/swrocksdb/seungsu/pororo/lib64/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [17]:
# ranking by cos sim group_by cluster id
for cid in range(len(cluster_id_set)):
    tmp = df[df.clusters == cid]
    mean = sum(list(tmp["vector"]))/len(list(tmp["vector"]))

    rank = []
    for idx in tmp.index:
        simularity = cos_sim(df.loc[idx, "vector"], mean)
        df.loc[idx, "cos_sim"] = simularity
        rank.append((idx, simularity))

    rank.sort(key=lambda x : x[1], reverse=True)

    ranking = 1
    for idx, v in rank:
        df.loc[idx, "rank"] = ranking
        df.loc[idx, "score"] = len(rank)/record_num
        # category_num_map['category']
        ranking += 1


In [18]:
# set cluster id not to overlap each other
try:
    pivot_id = db.get_max_clusterid()["max"] + 1
except:
    pivot_id = 1
df["clusters"] += pivot_id

In [19]:
# form data to update db
total_data = []
service_data = dict()
no_service_data = dict()

for i in df.index:
    if df["rank"][i] == 1:
        now = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        data = (int(df["clusters"][i]), "true", i)
        sdata = (i, df["created_datetime"][i].strftime('%Y-%m-%d %H:%M:%S'), now, df["article_context"][i], df["article_url"][i], df["category"][i], df["article_headline"][i], df["image_url"][i], df["score"][i], now, df["score"][i])
        service_data[i] = sdata
    else:
        ndata = (i, )
        no_service_data[i] = ndata
        data = (int(df["clusters"][i]), "false", i)

    total_data.append(data)


### 요약할 체크리스트 생성

In [20]:
check_list = list(map(lambda x: x, service_data.keys()))

In [21]:
check_list = dbs.check_already_serviced_data_by_news_id(check_list)

In [22]:
# summary, keyword extract 필요
check_list = list(map(lambda x : x.get("id"), check_list))

In [23]:
summ_data = [all_data[k] for k in check_list]

### 요약 및 키워드 추출

In [24]:
idx = 0
thread_num = 8
parallel_data_list = [[]for i in range(thread_num)]
while len(summ_data) != 0:
    parallel_data_list[idx%thread_num].append(summ_data.pop())
    idx += 1

In [25]:
%%time

STOPWORDS = ["한국일보", "매일경제", "일다", "이코노미스트", "조선일보", "국민일보", "MBN", "시사IN", "이데일리", "한경비즈니스", "서울신문", "월간 산", "더팩트", "헤럴드경제", "한국경제", "블로터", "SBS Biz", "뉴스1", "헬스조선", "디지털데일리", "YTN", "중앙SUNDAY", "코메디닷컴", "디지털타임스", "한겨레21", "노컷뉴스", "JTBC", "파이낸셜뉴스", "서울경제", "중앙일보", "세계일보", "매일신문", "TV조선", "동아일보", "기자협회보", "오마이뉴스", "한겨레", "매경이코노미", "경향신문", "한국경제TV", "여성신문", "강원일보", "코리아중앙데일리", "연합뉴스", "연합뉴스TV", "문화일보", "시사저널", "비즈니스워치", "신동아", "조세일보", "머니투데이", "레이디경향", "주간경향", "미디어오늘", "채널A", "부산일보", "프레시안", "동아사이언스", "코리아헤럴드", "ZDNet Korea", "SBS", "아이뉴스24", "뉴시스", "뉴스타파", "머니S", "데일리안", "전자신문", "조선비즈", "아시아경제", "MBC", "KBS", "주간조선", "주간동아"]
keyword_news_list = []
news_summary_list = []
lock = threading.Lock();
keyword_lock = threading.Lock();

thread_list = []
print("thread starting...")
for tmp_data in parallel_data_list:

    tmp_thread = threading.Thread(target=summarize_keyword, args=(tmp_data, keyword_news_list, news_summary_list, lock, keyword_lock, ), daemon=True)
    thread_list.append(tmp_thread)
    tmp_thread.start()

for th in thread_list:
    th.join()




thread starting...


  5%|▌         | 1/20 [00:00<00:03,  5.17it/s]

ES INSERT : {'title': '중앙시평 조공의 슬픈 기억이 어른거렸다', 'summary': '시진핑 주석이 추진하는 ‘일대일로’ 一帶一路·육해상 실크로드 경유 국가의 언론에 ‘감지 感知 중국’ Experience China 을 해보라는 행사에 초대됐다. 남중국해 인도양 지중해 남태평양을 연결하는 중국 남부의 해상 실크로드 一路 거점을 도는 8일간의 여정이었다. 광둥성 廣東省 광저우 廣州 ·주하이 珠海 ·후이저우 惠州 푸젠성 福建省 샤먼 廈門 ·취안저우 泉州 ·푸저우 福州 등 6개 도시를 2000㎞ 움직이며 둘러봤다.', 'article_url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=025&aid=0002966762', 'image_url': None, 'date': datetime.datetime(2020, 1, 10, 12, 49)}
KEYWORD UPSERT : [('2021-09-01 20:59:05', '2021-09-01 20:59:05', '시진핑', 'PERSON', '2021-09-01 20:59:05', '시진핑')]
ES INSERT : {'title': '그 영화 이 장면 기생충', 'summary': '기생충 ‘기생충’에서 ‘냄새’는 인상적인 메타포다. 동익의 표현에 의하면 “아주 오래된 무말랭이 냄새” 혹은 “행주 삶을 때 나는 냄새”다. 그 냄새를 감지하는 사람은 동익과 그의 아들 다솜 정현준 뿐이다.', 'article_url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=025&aid=0002966745', 'image_url': 'https://imgnews.pstatic.net/image/025/2020/01/10/0002966745_001_20200110061102134.jpg', 'date': datetime.datetime(2020, 1, 10, 1

 20%|██        | 4/20 [00:00<00:02,  7.21it/s]

ES INSERT : {'title': '사설 규제속도 못따라 가겠다는 中企 호소에 귀 막아선 안 된다', 'summary': '화관법·화평법에 노동 규제 겹쳐 폐업 위기 몰려 기업 현장애로 해소하는 데서 규제개혁 출발해야 환경 관련 규제가 날로 강화되면서 이에 대처할 시간이라도 달라는 중소기업들의 호소가 끊이지 않고 있다. 중소기업인들은 8일 조명래 환경부 장관 초청 간담회에서 “화학물질관리법 화관법 화학물질의 등록 및 평가에 관한 법률 화평법 등과 관련된 비용 급증으로 경영난이 심화되고 있다”며 정부가 적극 나서 부담을 줄여달라고 주문했다. 중소기업인들은 안전을 강화하자는 정부의 취지에는 공감하지만 기업 규모와 무관하게 일률적으로 밀어붙이는 것은 감당하기에 벅차다고 입을 모은다.', 'article_url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=110&oid=015&aid=0004271578', 'image_url': None, 'date': datetime.datetime(2020, 1, 10, 12, 9)}
KEYWORD UPSERT : [('2021-09-01 20:59:05', '2021-09-01 20:59:05', '조명래', 'PERSON', '2021-09-01 20:59:05', '조명래'), ('2021-09-01 20:59:05', '2021-09-01 20:59:05', '환경부', 'ORGANIZATION', '2021-09-01 20:59:05', '환경부')]
ES INSERT : {'title': 'CES2020 동양도 서양도 ‘잠이 보약’…ICT업계 “꿀잠 도와드립니다”', 'summary': '데이터수집→데이터활용 ‘진화’…안대·수면등·베게·이불·침대 “내가 적임자” 디지털데일리 윤상호기자 동서양을 막론하고 잠이 보약이다. 데이터 수집에서 데이터 활용으로 진화했다. 9일 현지시각 미국 라스베이거스에서 CES2020이 진행 중이다.', 'article_ur

...

ES INSERT : {'title': '게임 MCN ‘롤큐’ ‘2019 게임 크리에이터 파티’ 성료', 'summary': '머니투데이 중기 창업팀 게임 MCN ㈜롤큐 이하 롤큐 가 ‘2019 게임 크리에이터 파티’를 성황리에 종료했다고 밝혔다. 특히 본 행사는 내로라하는 게임 크리에이터들이 서로 친목을 도모하고 활발히 소통하며 국내 게임문화의 발전에 이바지하는 장으로 마련됐다는 점에서 의미를 지니고 있다. 게임 크리에이터들을 위한 파티인 만큼 ‘벤큐’의 첨단 게이밍 장비들을 체험 가능한 벤큐 부스를 선보이고 다양한 게임기들을 이용할 수 있는 환경을 조성해 참가자들로부터 호평을 받았다. 한편 롤큐는 2015년 설립된 국내 대표 게임 MCN이다.', 'article_url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=101&oid=008&aid=0004338245', 'image_url': 'https://imgnews.pstatic.net/image/008/2020/01/10/0004338245_001_20200110123201205.jpg', 'date': datetime.datetime(2020, 1, 10, 12, 31)}
ES INSERT : {'title': '공직자 속속 출사표 경쟁구도 가시화', 'summary': '사퇴시한 다가오며 행보 활발 유정배 석공사장 불출마 결정 4·15 총선을 앞두고 16일까지 사퇴 여부를 결정해야 하는 공직자들의 선택이 가시화되고 있다. 우선 더불어민주당으로 춘천 출마를 준비했던 유정배 대한석탄공사 사장이 최종적으로 불출마를 결정했다. 민주당 후보로 춘천 선거구에 출마할 예정이다.', 'article_url': 'https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=087&aid=0000783978', 'image_url': None, 'date': datetime.datetime(20








 95%|█████████▌| 19/20 [00:17<00:00,  1.07it/s]




100%|██████████| 20/20 [00:18<00:00,  1.11it/s]






100%|██████████| 20/20 [00:18<00:00,  1.11it/s]


KEYWORD UPSERT : [('2021-09-01 20:59:26', '2021-09-01 20:59:26', '벤큐', 'ORGANIZATION', '2021-09-01 20:59:26', '벤큐'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '롤큐', 'ORGANIZATION', '2021-09-01 20:59:26', '롤큐')]
KEYWORD UPSERT : [('2021-09-01 20:59:26', '2021-09-01 20:59:26', '유정배', 'PERSON', '2021-09-01 20:59:26', '유정배'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '대한석탄공사', 'ORGANIZATION', '2021-09-01 20:59:26', '대한석탄공사'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '민주당', 'ORGANIZATION', '2021-09-01 20:59:26', '민주당'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '더불어민주당', 'ORGANIZATION', '2021-09-01 20:59:26', '더불어민주당')]
KEYWORD UPSERT : [('2021-09-01 20:59:26', '2021-09-01 20:59:26', '강원도청', 'ARTIFACT', '2021-09-01 20:59:26', '강원도청'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '평화지역발전본부', 'ORGANIZATION', '2021-09-01 20:59:26', '평화지역발전본부'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '국방부', 'ORGANIZATION', '2021-09-01 20:59:26', '국방부'), ('2021-09-01 20:59:26', '20




 95%|█████████▌| 20/21 [00:18<00:00,  1.28it/s]






100%|██████████| 20/20 [00:17<00:00,  1.13it/s]



100%|██████████| 21/21 [00:18<00:00,  1.12it/s]

KEYWORD UPSERT : [('2021-09-01 20:59:26', '2021-09-01 20:59:26', '스카이컨벤션', 'ARTIFACT', '2021-09-01 20:59:26', '스카이컨벤션'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '한국당', 'ORGANIZATION', '2021-09-01 20:59:26', '한국당'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '이현종', 'PERSON', '2021-09-01 20:59:26', '이현종'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '황교안', 'PERSON', '2021-09-01 20:59:26', '황교안'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '더불어민주당', 'ORGANIZATION', '2021-09-01 20:59:26', '더불어민주당'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '최문순', 'PERSON', '2021-09-01 20:59:26', '최문순'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '신년인사회', 'EVENT', '2021-09-01 20:59:26', '신년인사회'), ('2021-09-01 20:59:26', '2021-09-01 20:59:26', '문재인', 'PERSON', '2021-09-01 20:59:26', '문재인')]
ES INSERT : {'title': '대학살 다음 날…이낙연까지 윤석열 협공', 'summary': '추미애·윤석열 인사협의 불발 관련 이 총리 “장관이 필요한 대응하라” 검찰 “법무부에 징계·감찰 요구한 것” 한국당 “검찰 숙청 망나니 정권” ‘윤석열 사단’의 해체 인사 다음 날 이낙연 국무총리까지 나서 검찰총장을 공개 압박했다. 이 총리

In [26]:
for v in news_summary_list:
    tmp = list(service_data[v[0]])
    tmp[3] = v[3]
    service_data[v[0]] = tuple(tmp)

In [27]:
# db.update_cid_representative(total_data)
print(total_data[:10])

# dbs.upsert_news(list(service_data.values()))
print(list(service_data.values())[:10])

# dbs.insert_keyword_news_relation(keyword_news_list)
print(keyword_news_list[:10])

# dbs.remove_news_by_id(list(no_service_data.values()))
print(list(no_service_data.values())[:10])

# dbs.remove_keyword_news_mapping_by_newsid(list(service_data.values()))
print(list(service_data.values())[:10])

[(2343885, 'true', 11054690000456330), (2343825, 'false', 11040010011327585), (2343886, 'true', 11001190002376052), (2343887, 'true', 11014170000491941), (2343888, 'true', 11020020002117915), (2343889, 'true', 11020010011327576), (2343890, 'true', 11040560010781295), (2343826, 'false', 11010320002985410), (2343827, 'false', 11004210004402369), (2343891, 'true', 11020140004355774)]
[(11054690000456330, '2020-01-10 13:05:00', '2021-09-01 20:58:28', '로이터 연합뉴스 “1990년대의 기술 플랫폼은 데스크톱 컴퓨터였고 2000년대는 웹이었으며 2010년대는 휴대폰이었습니다. 각 시대마다 이 플랫폼들은 점점 자연스럽게 우리의 일상에 스며들었죠. 앞으로도 여전히 휴대폰이 중요하겠지만 2020년대의 어느 시점이 되면 증강현실 AR 글래스가 우리와 기술의 관계를 다시 정의 내릴 겁니다.” 마크 저커버그 사진 페이스북 최고경영자 CEO 가 9일 현지시간 자신의 사회관계망서비스 SNS 인 페이스북에 2020년대 기술 플랫폼 전망을 이렇게 내다봤다. 저커버그는 “올해는 1년짜리 새해 다짐보다는 2030년에 나와 세상이 어떻게 돼 있을지에 좀 더 집중해 보려고 한다”면서 향후 10년간 주목해야 될 사안을 소개했다. 그는 우선 다음 세대 컴퓨팅 플랫폼으로 ‘AR글래스’를 주목했다. AR은 실제현실 위에 디지털로 구현한 가상현실을 포개서 띄우는 기술로 AR글래스는 이를 도와주는 안경 형태의 디지털 기기다. 저커버그는 “어디든 ‘현재’가 될 수 있는 이 기술은 우리 사회의 가장 큰 문제들을 해결해줄 수 있을 것”이라며 대표적인 사례로 

In [28]:
# update 테스트시에는 실행 X
# while True:
#     try:
#         db.update_cid_representative(total_data)
#         break
#     except Exception as e:
#         log("update_cluster_error.txt", e)
#         log("update_cluster_error.txt", "")
#         timelib.sleep(1)
#         db = DBController()

# while True:
#     try:
#         # 새롭게 선출된 뉴스 추가
#         dbs.upsert_news(list(service_data.values()))
#         break
#     except Exception as e:
#         log("upsert_repr_news_error.txt", e)
#         log("upsert_repr_news_error.txt", "")
#         timelib.sleep(1)
#         dbs = ServiceDBController()
# 키워드 매핑 관계 DB에 추가
# dbs.insert_keyword_news_relation(keyword_news_list)

# while True:
#     try:
#         # 탈락 뉴스 Service DB에서 삭제
#         dbs.remove_news_by_id(list(no_service_data.values()))
#         # 뉴스 키워드 매핑 정보까지 삭제
#         dbs.remove_keyword_news_mapping_by_newsid(list(service_data.values()))
#         break
#     except Exception as e:
#         log("remove_service_error.txt", e)
#         log("remove_service_error.txt", "")
#         timelib.sleep(1)
#         dbs = ServiceDBController()